In [5]:
import pickle
import torch
import pandas as pd
import numpy as np

import sys
sys.path.append("..")
import clustering

from transformers import BertTokenizer

/home/gsc685/.conda/envs/gabriella_cwr4lsc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
feature_set = 'binder'
#feature_set = 'buchanan'

get the clustering and usages that the clustering is based on 

In [7]:
with open('../data/cwr4lsc/{}/usages_len128.clustering.2.dict'.format(feature_set), 'rb') as f:
    clusterings = pickle.load(f)

In [8]:
with open('../data/cwr4lsc/{}/matrix_usages_16_len128.dict'.format(feature_set), 'rb') as f:
    usages = pickle.load(f)

In [9]:
words = usages.keys()


Let's take a look at some of our data, focusing ont he word virtual

In [16]:
word = 'virtual'

In [10]:
decades = list(set(usages['virtual'][3]))
decades

[1920, 1990, 1960, 1930, 2000, 1970, 1940, 1910, 1980, 1950]

Just to see, how many clusters do we have for 'virtual'

In [15]:
len(clusterings['virtual'].cluster_centers_)

2

In [27]:
set(clusterings['virtual'].labels_)

{0, 1}

In [33]:
U_w, contexts, positions, t_labels = usages['virtual']
print(len(U_w))
print(len(clusterings['virtual'].labels_))

1940
1940


In [17]:
def features_by_decade(feature_model, word, usages, clustering):
    """
    function that takes a clustering over a set of usages and finds the center of each usage type within each decade.
    that is, if there are three overall clusters for a word, it finds the center of all the usages in a single decade 
    that are in that cluster, and calculates *their* centroid. 
    """
    
    cluster_ids = range(0, len(clustering.cluster_centers_))
    
    U_w, contexts, positions, t_labels = usages
        
    decades = list(set(t_labels))
    
    feature_map = feature_model.feature_norms.feature_map
    feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]    
    
    results = []
    for decade in decades:
        
        usages_in_this_decade_indices = set(np.flatnonzero(t_labels == decade) ) # get the indexes in our usages of the labels in this decade
        
        for cluster_id in cluster_ids:
            
            cluster_indices = set(np.flatnonzero(clustering.labels_ == cluster_id) )
            
            this_decade_this_cluster = list(usages_in_this_decade_indices.intersection(cluster_indices))
            #print(this_decade_this_cluster)

            feature_vecs = np.array([ U_w[index] for index in this_decade_this_cluster])
            center = np.mean(feature_vecs, axis=0)
            #print(center.shape)
            #print(center[300])
            
            i=0
            for value in center:

                data = {
                    "word": word,
                    "decade": decade,
                    "cluster_id": cluster_id,
                    "feature": feature_labels[i],
                    "feature_value": value
                }
                results.append(data)
                i+=1

    return results

In [19]:
feature_model = buchanan = torch.load('../trained_models/{}'.format(feature_set))

In [20]:
"""
get a dataframe with features representing the center of a cluster in each decade. so, just the 1980s 'virtual_1', or just the 1930s 'virtual_0'. 
There might be none if there are no examples of a usage type in a given decade
"""

"\nget a dataframe with features representing the center of a cluster in each decade. so, just the 1980s 'virtual_1', or just the 1930s 'virtual_0'. \nThere might be none if there are no examples of a usage type in a given decade\n"

In [34]:
features = []
for word in words:
    features += features_by_decade(feature_model, word, usages[word], clusterings[word])

In [42]:
df = pd.DataFrame.from_records(features)
df.to_csv('../data/cwr4lsc/results/{}/by_decade_centroids.csv'.format(feature_set))

In [43]:
from IPython.display import display
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', None)  # Display full string without truncation
display(df)  

,word,decade,cluster_id,feature,feature_value
0,leaf,1920,0,Vision,4.691048
1,leaf,1920,0,Bright,1.916816
2,leaf,1920,0,Dark,1.184283
3,leaf,1920,0,Color,3.967469
4,leaf,1920,0,Pattern,3.504453
...,...,...,...,...,...
26655,optical,1950,1,Surprised,1.599815
26656,optical,1950,1,Drive,1.730034
26657,optical,1950,1,Needs,1.287219
26658,optical,1950,1,Attention,3.557145


In [45]:
# what kinds of attention features do we get?
df[df['feature']=='Attention'].sort_values(by=['cluster_id', 'decade']).head()

,word,decade,cluster_id,feature,feature_value
928,leaf,1910,0,Attention,0.326324
2168,sleep,1910,0,Attention,0.755797
3408,fire,1910,0,Attention,4.542366
4648,energy,1910,0,Attention,3.236839
5888,coach,1910,0,Attention,2.540640


In [ ]:
"""""
which features changed the most?
""""